In [1]:
import pandas as pd

In [7]:
# Top-Ten warmest countries
df_weather = pd.read_parquet('./../data/weather_compressed_new.parquet')
df_countries_extended = pd.read_csv('./../data/countries_extended.csv')[['country', 'continent']]
df_countries_weather_extended = pd.merge(df_countries_extended, df_weather, on='country')
dates = pd.read_csv('./../data/dates.csv')['date'].unique()
dates

array(['1960-01-01', '1960-01-15', '1960-01-29', ..., '2023-08-04',
       '2023-08-18', '2023-09-01'], dtype=object)

In [ ]:
# Top 5 coldest
scores = {}
df_countries_weather_extended['min_temp_c'] = df_countries_weather_extended['min_temp_c'].fillna(df_countries_weather_extended['avg_temp_c'])
df_countries_weather_extended = df_countries_weather_extended.dropna(subset=['min_temp_c'])

for date in dates:
    df_country_max_temp_c = df_countries_weather_extended[df_countries_weather_extended['date'] == str(date)][['country', 'min_temp_c']]
    for (i, (country, min_temp_c)) in df_country_max_temp_c.iterrows():
        scores[country] = scores.get(country, 0.0) + min_temp_c

df_score_smallest = pd.DataFrame.from_dict(data=scores, orient='index', columns=['score'])
countries_smallest = df_score_smallest.nsmallest(5, 'score').index

df_result_smallest = pd.DataFrame([], columns=df_countries_weather_extended[['date', 'country', 'min_temp_c']].columns)

for country in countries_smallest:
    for date in dates:
        temp = df_countries_weather_extended[(df_countries_weather_extended['date'] == str(date)) & (df_countries_weather_extended['country'] == country)][['date', 'country', 'min_temp_c']]
        if not temp.empty:
            df_result_smallest.loc[len(df_result_smallest)] = temp.iloc[0][['date', 'country', 'min_temp_c']]


In [4]:
# Top 5 hottest
scores = {}
df_countries_weather_extended['max_temp_c'] = df_countries_weather_extended['max_temp_c'].fillna(df_countries_weather_extended['avg_temp_c'])
df_countries_weather_extended = df_countries_weather_extended.dropna(subset=['max_temp_c'])

for date in dates:
    df_country_max_temp_c = df_countries_weather_extended[df_countries_weather_extended['date'] == str(date)][['country', 'max_temp_c']]
    for (i, (country, max_temp_c)) in df_country_max_temp_c.iterrows():
        scores[country] = scores.get(country, 0.0) + max_temp_c

df_score_hottest = pd.DataFrame.from_dict(data=scores, orient='index', columns=['score'])
countries_hottest = df_score_hottest.nlargest(5, 'score').index

df_result_hottest = pd.DataFrame([], columns=df_countries_weather_extended[['date', 'country', 'max_temp_c']].columns)

for country in countries_hottest:
    for date in dates:
        temp = df_countries_weather_extended[(df_countries_weather_extended['date'] == str(date)) & (df_countries_weather_extended['country'] == country)][['date', 'country', 'max_temp_c']]
        if not temp.empty:
            df_result_hottest.loc[len(df_result_hottest)] = temp.iloc[0][['date', 'country', 'max_temp_c']]


In [ ]:
df_result_hottest.to_csv('./data/country_overall/top_5_warmest_countries.csv')
df_result_smallest.to_csv('./data/country_overall/top_5_coldest_countries.csv')